## Notebook to convert Planet Digital Numbers to Surface Reflectance values, then stitch all images captured on the same day

Rainey Aberle & Jukes Liu

January 2022

In [1]:
import os
import rasterio as rio
import numpy as np
import glob
import subprocess
from osgeo import gdal
from xml.dom import minidom

In [2]:
### ---path to images--- ###
basepath = '/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/PSScene4Band/'

### ---path for output images--- ###
outpath = '/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/'

### ---extension for mosaicing--- ###
ext = 'SR_clip'

### Convert image radiance values to surface reflectance, save as geotiff

In [3]:
# make directory for SR image outputs
SRpath = outpath+'SR/'
if os.path.isdir(SRpath)==0:
    os.mkdir(SRpath)
    print(SRpath+' directory made')

# grab image and metadata file names
os.chdir(basepath)
im_names = glob.glob('*'+ext+'.tif')

# loop through images
for im_name in im_names:
    
    # open image 
    im = rio.open(im_name)
    # read bands
    blue = im.read(1)
    green = im.read(2)
    red = im.read(3)
    nir = im.read(4)
    
    # open metadata
    datestring = im_name[0:15] # match date string from image file
    im_meta_name = glob.glob(datestring+'*metadata*.xml')
    im_meta = minidom.parse(im_meta_name[0])
    nodes = im_meta.getElementsByTagName("ps:bandSpecificMetadata")

    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)

    # Multiply the Digital Number (DN) values in each band by the TOA reflectance coefficients
    # Include a scaling factor of 10000 to save values as integers
    scalar = 10000
    blue_sr = blue * coeffs[1] * scalar
    green_sr = green * coeffs[2] * scalar
    red_sr = red * coeffs[3] * scalar
    nir_sr = nir * coeffs[4] * scalar
    
    # print range
#     print(im_name)
#     print('b_sr: ', np.amin(blue_sr), np.amax(blue_sr))
#     print('g_sr: ', np.amin(green_sr), np.amax(green_sr))
#     print('r_sr: ', np.amin(red_sr), np.amax(red_sr))
#     print('nir_sr: ', np.amin(nir_sr), np.amax(nir_sr))
#     print('')
    
    # Set spatial characteristics of the output object to mirror the input
    kwargs = im.meta
    kwargs.update(count = 4, dtype=rio.uint16, driver='GTiff')
    
    # Write SR bands to a new raster file    
    with rio.open(outpath+'SR/'+datestring+'_SR.tif', 'w', **kwargs) as dst:
        dst.write_band(1, blue_sr.astype(rio.uint16))
        dst.write_band(2, green_sr.astype(rio.uint16))
        dst.write_band(3, red_sr.astype(rio.uint16))
        dst.write_band(4, nir_sr.astype(rio.uint16))
        print(outpath+'SR/'+datestring+'_SR.tif saved')
    

/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_203130_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210427_194054_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210714_204343_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210704_204515_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210518_211339_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210701_203411_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210717_213412_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210427_194055_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210821_204640_SR.ti

/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210606_202719_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210731_204802_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_170008_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210615_211616_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210815_204453_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210731_201802_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210719_205616_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210611_193904_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_165658_SR.ti

/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210704_204640_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210424_202443_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210524_165520_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_165515_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_203132_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210613_202526_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210818_204909_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210517_202036_SR.tif saved
/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210801_164621_SR.ti

### Mosaic reflectance image files with a specified extension

In [4]:
uniquescenes = [] # grab unique scenes from a folder with all scenefiles
for scene in os.listdir(SRpath):
    date = scene[0:8]
    uniquescenes.append(date)
uniquescenes = list(set(uniquescenes))
uniquescenes.sort() # sort chronologically
uniquescenes

['20210421',
 '20210422',
 '20210423',
 '20210424',
 '20210425',
 '20210426',
 '20210427',
 '20210429',
 '20210501',
 '20210507',
 '20210517',
 '20210518',
 '20210524',
 '20210605',
 '20210606',
 '20210611',
 '20210612',
 '20210613',
 '20210614',
 '20210615',
 '20210629',
 '20210630',
 '20210701',
 '20210704',
 '20210714',
 '20210715',
 '20210716',
 '20210717',
 '20210718',
 '20210719',
 '20210726',
 '20210731',
 '20210801',
 '20210802',
 '20210813',
 '20210815',
 '20210816',
 '20210818',
 '20210821']

In [5]:
# make directory for SR stitched outputs
SRstitchedpath = outpath+'SR-stitched/'
if os.path.isdir(SRstitchedpath)==0:
    os.mkdir(SRstitchedpath)
    print(SRstitchedpath+' directory made')

# loop through unique scenes
for scene in uniquescenes:
    filepaths = [] # files from the same date to mosaic together
    for file in os.listdir(SRpath): # check all files
        if file.startswith(scene): # if they match the scene date
            filepaths.append(SRpath+file) # add the path to the file
    
    # construct the gdal_merge command
    cmd = 'gdal_merge.py -v '

    # add in input files
    for filepath in filepaths:
        cmd += filepath+' '
    
    # define the out path with correct extension
    if ext == 'DN_udm':
        out = os.path.join(SRstitchedpath, scene + "_DN_mask.tif")
    elif ext == 'udm2':
        out = os.path.join(SRstitchedpath, scene + "_mask.tif")
    else:
        out = os.path.join(SRstitchedpath, scene + ".tif")

    cmd += '-o '+out

    # Run the command 
    p = subprocess.run(cmd, shell=True, capture_output=True) 
    print(p)

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_165700_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_165659_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_165658_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_211435_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_211433_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_202734_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210421.tif', returncode=0, stdout=b'\nProcessing file     1 of     6,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210421_165700_SR.tif\nFile Si

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_170327_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_203015_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_170329_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_203130_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_203018_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_170328_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210426_203132_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210426.tif', returncode=0, stdout=b'\nProcessing file     1 of     7,  0.000% completed in 0 minutes.\nFilename: /Users/r

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_165516_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_165517_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_170008_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_165515_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210507.tif', returncode=0, stdout=b'\nProcessing file     1 of     4,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210507_165516_SR.tif\nFile Size: 2650x2197x4\nPixel Size: 3.000000 x -3.000000\nUL:(391020.000000,6698451.000000)   LR:(398970.000000,6691860.000000)\nCopy 0,0,2650,2197 to 0,1719,2650,2197.\nCopy 0,0,2650,2197 to 0,1719,2650,2197.\nCopy 0,0

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210606_202722_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210606_202719_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210606_201946_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210606_201944_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210606.tif', returncode=0, stdout=b'\nProcessing file     1 of     4,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210606_202722_SR.tif\nFile Size: 643x3860x4\nPixel Size: 3.000000 x -3.000000\nUL:(391020.000000,6703608.000000)   LR:(392949.000000,6692028.000000)\nCopy 0,0,643,3860 to 0,0,643,3860.\nCopy 0,0,643,3860 to 0,0,643,3860.\nCopy 0,0,643,3860 t

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210614_205247_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210614_164759_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210614_164801_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210614_164758_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210614_164800_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210614.tif', returncode=0, stdout=b'\nProcessing file     1 of     5,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210614_205247_SR.tif\nFile Size: 712x1403x4\nPixel Size: 3.000000 x -3.000000\nUL:(391239.000000,6703608.000000)   LR:(393375.000000,66

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210630_205214_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210630.tif', returncode=0, stdout=b'\nProcessing file     1 of     1,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210630_205214_SR.tif\nFile Size: 2694x3781x4\nPixel Size: 3.000000 x -3.000000\nUL:(391020.000000,6703203.000000)   LR:(399102.000000,6691860.000000)\nCopy 0,0,2694,3781 to 0,0,2694,3781.\nCopy 0,0,2694,3781 to 0,0,2694,3781.\nCopy 0,0,2694,3781 to 0,0,2694,3781.\nCopy 0,0,2694,3781 to 0,0,2694,3781.\n', stderr=b'')
CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210701_203411_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210701_203

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210715_211159_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210715_204552_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210715_211156_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210715_204551_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210715_204553_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210715.tif', returncode=0, stdout=b'\nProcessing file     1 of     5,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210715_211159_SR.tif\nFile Size: 1996x3129x4\nPixel Size: 3.000000 x -3.000000\nUL:(393060.000000,6701247.000000)   LR:(399048.000000,6

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210719_205616_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210719.tif', returncode=0, stdout=b'\nProcessing file     1 of     1,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210719_205616_SR.tif\nFile Size: 2696x3916x4\nPixel Size: 3.000000 x -3.000000\nUL:(391020.000000,6703608.000000)   LR:(399108.000000,6691860.000000)\nCopy 0,0,2696,3916 to 0,0,2696,3916.\nCopy 0,0,2696,3916 to 0,0,2696,3916.\nCopy 0,0,2696,3916 to 0,0,2696,3916.\nCopy 0,0,2696,3916 to 0,0,2696,3916.\n', stderr=b'')
CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210726_204317_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210726_164

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_203626_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_204542_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_203624_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_204543_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_213636_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_213634_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210802_204541_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210802.tif', returncode=0, stdout=b'\nProcessing file     1 of     7,  0.000% completed in 0 minutes.\nFilename: /Users/r

CompletedProcess(args='gdal_merge.py -v /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210816_202318_SR.tif /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210816_202316_SR.tif -o /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR-stitched/20210816.tif', returncode=0, stdout=b'\nProcessing file     1 of     2,  0.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR/20210816_202318_SR.tif\nFile Size: 572x3858x4\nPixel Size: 3.000000 x -3.000000\nUL:(391020.000000,6703608.000000)   LR:(392736.000000,6692034.000000)\nCopy 0,0,572,3858 to 0,0,572,3858.\nCopy 0,0,572,3858 to 0,0,572,3858.\nCopy 0,0,572,3858 to 0,0,572,3858.\nCopy 0,0,572,3858 to 0,0,572,3858.\n\nProcessing file     2 of     2, 50.000% completed in 0 minutes.\nFilename: /Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/SR

In [6]:
# Print files in outpath to verify
os.listdir(SRstitchedpath)

['20210518.tif',
 '20210524.tif',
 '20210726.tif',
 '20210719.tif',
 '20210731.tif',
 '20210718.tif',
 '20210802.tif',
 '20210816.tif',
 '20210427.tif',
 '20210426.tif',
 '20210630.tif',
 '20210815.tif',
 '20210801.tif',
 '20210424.tif',
 '20210425.tif',
 '20210421.tif',
 '20210813.tif',
 '20210422.tif',
 '20210423.tif',
 '20210611.tif',
 '20210605.tif',
 '20210613.tif',
 '20210606.tif',
 '20210612.tif',
 '20210821.tif',
 '20210429.tif',
 '20210818.tif',
 '20210615.tif',
 '20210629.tif',
 '20210614.tif',
 '20210704.tif',
 '20210507.tif',
 '20210517.tif',
 '20210701.tif',
 '20210715.tif',
 '20210714.tif',
 '20210716.tif',
 '20210717.tif',
 '20210501.tif']

#### Done!